In [5]:
import os
import argparse
import json
import shutil
import sys
import pickle

import numpy as np
import torch
import skvideo.io

from numpy.lib.format import open_memmap

# sys.path.append(
#     os.path.abspath(os.path.join(os.path.dirname(__file__), os.path.pardir)))
sys.path.append(
    os.path.abspath(".."))
import tools
import tools.utils as utils


In [16]:

class Struct():
    pass
arg=Struct()
arg.project_path="/home/jiangdong/workspace/st-gcn/"
arg.data_path=os.path.join(arg.project_path,"output/Skating/")
part = ['train', 'val']
for p in part:
    json_file=os.path.join(arg.data_path,"{}_label.json".format(p))
    new_json_file=os.path.join(arg.data_path,"new_{}_label.json".format(p))
    with open(json_file,"r") as jf:
        videos_info=json.load(jf)
        new_json={}
        target_dir=os.path.join(arg.data_path,"{}_data".format(p))
        shutil.rmtree(target_dir, ignore_errors=True)
        if(os.path.exists(target_dir)==False):
            os.makedirs(target_dir)
        for name,value in videos_info.items():
            name=name.split('.')[0]
            old_path=os.path.join(arg.data_path,"data","{}.json".format(name))
            new_json[name]=value
#             print(old_path)
            target_path=os.path.join(target_dir,"{}.json".format(name))
#             print(target_path)
            command="cp {} {}".format(old_path,target_path)
            os.system(command)
        with open(new_json_file,'w') as f:
            json.dump(new_json,f)

In [3]:
import os
import json
class Struct():
    pass
arg=Struct()
arg.project_path="/home/jiangdong/workspace/st-gcn/"
arg.data_path=os.path.join(arg.project_path,"output/Skating/")
part = ['train', 'val']
for p in part:
    json_file=os.path.join(arg.data_path,"{}_label.json".format(p))
    new_json_file=os.path.join(arg.data_path,"new_{}_label.json".format(p))
    with open(json_file,"r") as jf:
        videos_info=json.load(jf)
        new_json={}
        for name,value in videos_info.items():
            name=name.split('.')[0]
            new_json[name]=value
        with open(new_json_file,'w') as f:
            json.dump(new_json,f)

In [3]:
import os
class Struct():
    pass
arg=Struct()
arg.data_path="../data/Skating"

arg.trainfile=os.path.join(arg.data_path,"train.csv")
arg.testfile=os.path.join(arg.data_path,"val.csv")
arg.labelfile=os.path.join(arg.data_path,"classInd.csv")
def _find_max(filename):
    with open(filename) as f:
        max_frames=0
        sum_frames=0
        count=0
        for line in f.readlines():
            info=line.strip()
            video_name ,frames,label=info.split(" ")
            sum_frames+=int(frames)
            count+=1
            max_frames=max(int(frames),max_frames)
        return max_frames,sum_frames,count,sum_frames/count
print(_find_max(arg.testfile))

(1326, 125139, 425, 294.44470588235293)


In [5]:
import numpy as np
import scipy.io as io 
import os
import pickle
import pandas as pd
class Struct():
    pass
arg=Struct()
p="train"
arg.project_path="/home/jiangdong/workspace/st-gcn/"
arg.data_path=os.path.join(arg.project_path,"output/Skating/")
datafile=os.path.join(arg.data_path,"{}_data.npy".format(p))
labelfile=os.path.join(arg.data_path,"{}_label.pkl".format(p))
matfile=os.path.join(arg.data_path,"{}.mat".format(p))
with open(labelfile,'rb') as lf:
    name,label=pickle.load(lf)
data = np.load(datafile)
# print(data)
# print(label)
io.savemat(matfile,{'data':data,'label':label,'name':name})
print(data.shape)

(989, 3, 1500, 18, 1)


# 思路整理
根据train.txt和val.txt获取每一个视频文件路径

对每个视频

    使用openpose生成预测文件
    
    将预测文件打包为数据和标签
    
    将数据和标签存入对应文件中

In [2]:
# class Struct():
#     pass
# arg=Struct()
# arg.data_path="../data/Skating"
# arg.out_folder="../output/Skating"

# arg.trainfile=os.path.join(arg.data_path,"train.csv")
# arg.testfile=os.path.join(arg.data_path,"val.csv")
# arg.labelfile=os.path.join(arg.data_path,"classInd.csv")

# arg.openpose="/home/jiangdong/opt/openpose/build"
# arg.model_folder="../models"
# openpose = '{}/examples/openpose/openpose.bin'.format(arg.openpose)

# def _count_lines(filename):
#     with open(filename) as f:
#         count=-1
#         for count,_ in enumerate(f):
#             pass
#         count+=1
#     return count

# def _video_loader(filename):
#     with open(filename) as f:
#         for line in f.readlines():
#             info=line.strip()
#             video_name , _,label=info.split(" ")
#             yield video_name,str(int(label)+1)

# def pose_estimation(openpose,out_folder,video_path,model_folder,info):
#     video_name=video_path.split('/')[-1].split('.')[0]
#     output_snippets_dir=os.path.join(out_folder,'openpose_estimation/snippets/{}'.format(video_name))
#     output_sequence_dir = os.path.join(out_folder,'data/')
#     output_sequence_path = '{}/{}.json'.format(output_sequence_dir, video_name)
#     # pose estimation
#     openpose_args = dict(
#         video=video_path,
#         write_json=output_snippets_dir,
#         display=0,
#         render_pose=0, 
#         model_pose='COCO',
#         model_folder=model_folder)
#     command_line = openpose + ' '
#     command_line += ' '.join(['--{} {}'.format(k, v) for k, v in openpose_args.items()])
#     shutil.rmtree(output_snippets_dir, ignore_errors=True)
#     os.makedirs(output_snippets_dir)
#     print(command_line)
#     os.system(command_line)
#     # pack openpose ouputs
#     video = utils.video.get_video_frames(video_path)
#     height, width, _ = video[0].shape
#     video_info = utils.openpose.json_pack(
#         output_snippets_dir, video_name, width, height, label_index=info["label_index"],label=info["label"])
#     if not os.path.exists(output_sequence_dir):
#         os.makedirs(output_sequence_dir)
#     with open(output_sequence_path, 'w') as outfile:
#         json.dump(video_info, outfile)
#     if len(video_info['data']) == 0:
#         print('Can not find pose estimation results of %s'%(video_name))
#         return
#     else:
#         print('%s Pose estimation complete.'%(video_name))    
# print(os.getcwd())
# label_names={}
# with open(arg.labelfile) as lf:
#     for line in lf.readlines():
#         index,label_name=line.strip().split(" ")
#         label_names[index]=label_name
# print(label_names)
# part = ['train', 'val']
# for p in part:
#     csvfile=os.path.join(arg.data_path,"{}.csv".format(p))
#     label_file={}
#     total_count = _count_lines(csvfile)
#     count=0
#     for nameinfo,label in _video_loader(csvfile):
#         try:
#             filename=nameinfo.split('/')[3]+".mp4"
#             category=filename.split("_")[0]
#             info={}
#             info['label_index']=int(label)
#             info['has_skeleton']=True
#             info['label']=label_names[label]
#             label_file[filename]=info
#             video_path = os.path.join(arg.data_path,category,filename)
#             pose_estimation(openpose,arg.out_folder,video_path,arg.model_folder,info)
#             count+=1
#             print("%4.2f %% of %s has been processed"%(count*100/total_count,p))
#         except Exception as e:
#             print(e)
#     label_save_path=os.path.join(arg.out_folder,"{}_label.json".format(p))
#     with open(label_save_path,"w") as f:
#         json.dump(label_file,f)

In [ ]:
line="/share/SkatingFlow/3Lutz_n28_p10_g04"
print(line.split('/')[3].split("_")[0])